### Importing 

In [5]:
import sqlite3
import requests
import re
from bs4 import BeautifulSoup, Comment
import json
import pandas as pd
import time
import folium
import matplotlib.pyplot as plt
import urllib.request
from selenium import webdriver
import random
from xgboost import XGBClassifier
import traceback

In [6]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, f1_score, recall_score

In [7]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #WBOT
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer #these three are for visualization
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn import metrics
from matplotlib import cm
import numpy as np




import string, re
from nltk.stem import WordNetLemmatizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GMoneyMan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GMoneyMan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GMoneyMan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

### Cleaning the Dataframe

In [9]:
#read in the csv
beer_df = pd.read_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_w_reviews.csv')

In [10]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url,review
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,NaN,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev..."
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,NaN,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,NaN,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%..."
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.20,NaN,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,NaN,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...
...,...,...,...,...,...,...,...,...,...
7339,Fruit and Field,Radler,Occy's Brewery,4.80,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/2128...,NaN
7340,Fruit and Field,Alan-A-Dale Apple Ale,Robin Hood Brewing Co. / Home D Pizzeria,5.00,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/4182...,NaN
7341,Fruit and Field,Tussle In the Hay,Back Unturned Brewing Company,7.90,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/5844...,NaN
7342,Fruit and Field,Strawberry Chocolate Cheesecake Sour,Barn Town Brewing,5.90,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/4760...,NaN


In [11]:
#check if there are any NaN values
beer_df.isna().sum()

beer_style         0
beer_name          0
brewery_name       0
abv              144
no_ratings       144
avg_rating       144
drop_me         7344
url              144
review          1799
dtype: int64

In [12]:
#dropping the column that has no data
beer_df.drop('drop_me', axis = 1, inplace = True)

In [13]:
#dropping the rows that have no reviews
beer_df.dropna(subset=['review'], inplace = True)

In [14]:
#checking if there are any reviewed beers without ratings
empty_rating = beer_df[beer_df['no_ratings'] == 0]

In [15]:
#turn this column from str to float
beer_df['no_ratings'] = beer_df['no_ratings'].str.replace(',', '').astype(float)

In [16]:
#dropping the columns without ratings
beer_df.drop(empty_rating.index, inplace = True)

### Cleaning the Reviews w/ Stopwords

In [17]:
#stopwords and punctuation
stop_words = stopwords.words('english')
stop_words.extend(['look', 'smell', 'taste', 'feel', 'overall', 'rdev', 'beer', 'beers','nice','adams', 'jan', 'feb', 'mar',
                  'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'canada', 'north'])
stop_words += string.punctuation

In [18]:
lemmatizer = WordNetLemmatizer()

In [19]:
# A function that will apply stop words, remove numbers, punctuation, and normalization to a body of text
def pre_process_text(text):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    def clean(doc):
        if doc is not None:
            stop_free = " ".join([i for i in doc.lower().split() if i not in stop and i.isalpha()])
            punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
            normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
            return normalized
        else:
            pass
    processed_text = clean(text)
    if processed_text is not None:
        return processed_text.split()

In [20]:
#takes in a dataframe with a column with raw text and outputs a new column with pre-processed text
def add_processed_text(df, output_column_name, column_to_target='review'):
    df[output_column_name] = df.apply(lambda row: pre_process_text(row[column_to_target]), axis=1)

In [21]:
def count_word_frequency(vectorizer, text):
    try:
        termFrequencyMatrix = vectorizer.fit_transform(text)
        return termFrequencyMatrix
    except:
        print('Problem in count word freq')
        print(traceback.format_exc())

In [22]:
#runs a vectorizer over a row of the target column and if empty it will pass. Afterwards it will 
#generate a new column with the term frequency matrix
def get_term_freq_matrices(vectorizer, df, target_column):
    df['tf_matrix'] = df.apply(lambda row: count_word_frequency(vectorizer, row[target_column]) if
    row[target_column] != None else None, axis=1)

In [23]:
#runs the processor over the review column and creates a new one with the results
add_processed_text(beer_df, 'clean_reviews', column_to_target='review')

In [24]:
#runs the vectorizer over the df and column we just created with the processed text
tfidf = TfidfVectorizer()
get_term_freq_matrices(tfidf, beer_df, 'clean_reviews')

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

...

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix = vectorizer.fit_transform(text)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1841, in fit_transform
    X = super().fit_transform(raw_documents)
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1198, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents,
  File "C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\sklearn\feature_extraction\text.py", line 1129, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words

Problem in count word freq
Traceback (most recent call last):
  File "<ipython-input-21-29e726eba38e>", line 3, in count_word_frequency
    termFrequencyMatrix 

In [25]:
#creates a column that breaks the text out of a list and into a str
beer_df['naked_reviews'] = beer_df.apply(lambda row: ' '.join(row['clean_reviews']), axis =1)

In [26]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review,clean_reviews,tf_matrix,naked_reviews
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,5915.0,3.42,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev...","[pours, little, cloudy, little, small, smell, ...","(0, 276)\t1.0\n (1, 213)\t1.0\n (2, 63)\t1...",pours little cloudy little small smell like fr...
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029.0,3.19,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...,"[bottle, shaker, pint, hazy, copper, thick, wh...","(0, 44)\t1.0\n (1, 324)\t1.0\n (2, 279)\t1...",bottle shaker pint hazy copper thick white che...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,3992.0,4.41,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%...","[beautiful, ruby, red, head, dissipates, smell...","(0, 37)\t1.0\n (1, 435)\t1.0\n (2, 416)\t1...",beautiful ruby red head dissipates smell inten...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841.0,3.20,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...,"[one, favorite, summer, beer, smell, taste, bi...","(0, 347)\t1.0\n (1, 176)\t1.0\n (2, 484)\t...",one favorite summer beer smell taste bit like ...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,3799.0,3.91,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...,"[pour, pink, raspberry, impressive, white, hea...","(0, 299)\t1.0\n (1, 293)\t1.0\n (2, 312)\t...",pour pink raspberry impressive white head ligh...
...,...,...,...,...,...,...,...,...,...,...,...
5651,Fruit and Field,Carney Juice,Independent Republic Brewing Co.,4.00,1.0,4.00,https://www.beeradvocate.com/beer/profile/5671...,['PaleAleWalt from New York4/5\xa0\xa0rDev 0%l...,"[new, shandy, hint]","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 0)\t1.0",new shandy hint
5652,Fruit and Field,Cherry Ace,The Blue Elephant Craft Brew House,4.80,1.0,3.31,https://www.beeradvocate.com/beer/profile/3446...,['KrisDLSmith from Canada (ON)3.31/5\xa0\xa0rD...,[canada],"(0, 0)\t1.0",canada
5653,Fruit and Field,Cherry Stout,Cottontown Brew Lab,?,1.0,3.41,https://www.beeradvocate.com/beer/profile/5210...,['RochefortChris from North Carolina3.41/5\xa0...,[north],"(0, 0)\t1.0",north
5863,Fruit and Field,Cherry & Blackcurrant Wheat,Finkel & Garf Brewing Company,4.80,0.0,0.00,https://www.beeradvocate.com/beer/profile/3846...,['KT3418 from Colorado3/5\xa0\xa0rDev 0%look: ...,[],None,


In [27]:
#turned empty lists into nan values after stopwords
beer_df.loc[~beer_df.clean_reviews.astype(bool),'clean_reviews']=np.nan

In [28]:
#checked the rows that were now empty
beer_df.isna().sum()

beer_style          0
beer_name           0
brewery_name        0
abv                 0
no_ratings          0
avg_rating          0
url                 0
review              0
clean_reviews    1468
tf_matrix        1468
naked_reviews       0
dtype: int64

In [29]:
#dropped the rows with nan values
beer_df.dropna(subset=['clean_reviews'], inplace = True)

In [30]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review,clean_reviews,tf_matrix,naked_reviews
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,5915.0,3.42,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev...","[pours, little, cloudy, little, small, smell, ...","(0, 276)\t1.0\n (1, 213)\t1.0\n (2, 63)\t1...",pours little cloudy little small smell like fr...
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029.0,3.19,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...,"[bottle, shaker, pint, hazy, copper, thick, wh...","(0, 44)\t1.0\n (1, 324)\t1.0\n (2, 279)\t1...",bottle shaker pint hazy copper thick white che...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,3992.0,4.41,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%...","[beautiful, ruby, red, head, dissipates, smell...","(0, 37)\t1.0\n (1, 435)\t1.0\n (2, 416)\t1...",beautiful ruby red head dissipates smell inten...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841.0,3.20,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...,"[one, favorite, summer, beer, smell, taste, bi...","(0, 347)\t1.0\n (1, 176)\t1.0\n (2, 484)\t...",one favorite summer beer smell taste bit like ...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,3799.0,3.91,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...,"[pour, pink, raspberry, impressive, white, hea...","(0, 299)\t1.0\n (1, 293)\t1.0\n (2, 312)\t...",pour pink raspberry impressive white head ligh...
...,...,...,...,...,...,...,...,...,...,...,...
5639,Fruit and Field,Mos'ka Garnatxa Beer,Cerveseria Birrart,6.00,1.0,3.67,https://www.beeradvocate.com/beer/profile/1953...,['GONZALOYANNA from Spain3.67/5\xa0\xa0rDev 0%...,"[timmermans, flute, enjoyed, reddish, amber, f...","(0, 50)\t1.0\n (1, 13)\t1.0\n (2, 8)\t1.0\...",timmermans flute enjoyed reddish amber foggy f...
5648,Fruit and Field,Rainier Raspberry Wheat,Harmon Restaurant & Brewery,?,1.0,2.60,https://www.beeradvocate.com/beer/profile/1386...,['barleywinefiend from Washington2.6/5\xa0\xa0...,"[poured, cloudy, straw, color, almost, head, w...","(0, 8)\t1.0\n (1, 1)\t1.0\n (2, 11)\t1.0\n...",poured cloudy straw color almost head white ma...
5651,Fruit and Field,Carney Juice,Independent Republic Brewing Co.,4.00,1.0,4.00,https://www.beeradvocate.com/beer/profile/5671...,['PaleAleWalt from New York4/5\xa0\xa0rDev 0%l...,"[new, shandy, hint]","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 0)\t1.0",new shandy hint
5652,Fruit and Field,Cherry Ace,The Blue Elephant Craft Brew House,4.80,1.0,3.31,https://www.beeradvocate.com/beer/profile/3446...,['KrisDLSmith from Canada (ON)3.31/5\xa0\xa0rD...,[canada],"(0, 0)\t1.0",canada


### Modeling

In [32]:
beer_df['avg_rating'] = beer_df['avg_rating'].round()

In [33]:
beer_df['avg_rating'].value_counts()

4.0    2534
3.0    1270
2.0     196
5.0      66
1.0      11
Name: avg_rating, dtype: int64

In [34]:
x = beer_df['naked_reviews']
y = beer_df['avg_rating']

In [35]:
#running train / test / split on naked reviews (str)
data_train, data_test, target_train, target_test = train_test_split(x, y, test_size=0.20, random_state=21)

In [36]:
tfidf = TfidfVectorizer(max_df = .9, min_df=.1)

In [37]:
tfidf_data_train = tfidf.fit_transform(data_train)
tfidf_data_test = tfidf.transform(data_test)

print(tfidf_data_train.shape)
print(tfidf_data_test.shape)

(3261, 226)
(816, 226)


In [38]:
non_zero_cols = tfidf_data_train.nnz / float(tfidf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tfidf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 42.61729530818767
Percentage of columns containing 0: 0.8114278968664262


#### RandomForest (basic)

In [39]:
rf = RandomForestClassifier(random_state = 21)#max_depth=10,max_features=10)
rf.fit(tfidf_data_train, target_train)

RandomForestClassifier(random_state=21)

In [40]:
rf_test_pred = rf.predict(tfidf_data_test)

In [41]:
rf_recall = recall_score(target_test, rf_test_pred, average='weighted')
rf_acc_score = accuracy_score(target_test, rf_test_pred)
rf_f1_score = f1_score(target_test, rf_test_pred, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(rf_recall))
print("Testing Accuracy: {:.4}".format(rf_acc_score))
print("F1 Score: {:.4}".format(rf_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.6752
Testing Accuracy: 0.6752
F1 Score: 0.6178


#### XGBoost (basic)

In [42]:
xgb = XGBClassifier()
xgb.fit(tfidf_data_train, target_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [43]:
xgb_test_pred = xgb.predict(tfidf_data_test)

In [44]:
xgb_recall = recall_score(target_test, xgb_test_pred, average='weighted')
xgb_acc_score = accuracy_score(target_test, xgb_test_pred)
xgb_f1_score = f1_score(target_test, xgb_test_pred, average='weighted')

print('XGBOOST with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(xgb_recall))

print("Testing Accuracy: {:.4}".format(xgb_acc_score))
print("F1 Score: {:.4}".format(xgb_f1_score))

XGBOOST with Stematized Features:
---------------
Recall: 0.6875
Testing Accuracy: 0.6875
F1 Score: 0.6576


#### LogisticRegression (basic)

In [45]:
logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(tfidf_data_train, target_train)

In [46]:
log_test_pred = model_log.predict(tfidf_data_test)

In [47]:
log_recall = recall_score(target_test, log_test_pred, average='weighted')
log_acc_score = accuracy_score(target_test, log_test_pred)
log_f1_score = f1_score(target_test, log_test_pred, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(log_recall))

print("Testing Accuracy: {:.4}".format(log_acc_score))
print("F1 Score: {:.4}".format(log_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.6091
Testing Accuracy: 0.6091
F1 Score: 0.6231
